![Header](assets/header_advanced-3.svg)

# Advanced ③

Il nous reste plus que quelques choses à voir afin d'être opérationnel dans l'usage du langage et de tout ce qui l'entoure.

Après l'étude de quelques bibliothèques standard majeures puis l'installation d'une première bibliothèque tierce, vous devriez être capable de créer des projets Python de plus grande ampleur.

# Notions de ce cours

* 🖊️ La levée d'exceptions avec `raise`
* 🖊️ La levée d'exceptions de debug avec `assert`
* 🖊️ Set et frozenset
* 🔨 ️Lire et manipuler un set
* 🧰 Bibliothèque standard - `os.path`
* 🧰 Bibliothèque standard - `datetime`
* 🧰 Bibliothèque standard - `json`
* ⚙️ Installer un paquet avec PIP
* ⚙️ Les environnements virtuels
* 📚 Bibliothèque tierce - Pillow

---

## 🖊️ La levée d'exceptions avec `raise`

Après avoir vu comment **attraper** les exceptions qui peuvent se produire lors de l'exécution de notre code, il est temps d'apprendre à créer nos propres exceptions. Lorsqu'il arrive une erreur suffisamment importante pour vouloir la "remonter", c'est à dire la "propager" à travers notre code, on dit que l'on va "lever" (raise) une exception.

Voyons d'abord un exemple de code ayant une **expression booléenne** qui détermine si quelque chose doit s'exécuter ou non. Ici, l'application "appareil photo" d'un téléphone vérifie s'il ne fait pas trop chaud, car la prise de vidéos en haute résolution peut vite entraîner une chauffe du téléphone avec de lourds dégâts. 

In [10]:
print("Ouverture de l'appareil photo du téléphone.")
temperature = 42

if temperature >= 40:
    print("La température extérieure est trop élevée.")
else:
    print("Cheese !")

Ouverture de l'appareil photo du téléphone.
La température extérieure est trop élevée.


L'**expression booléenne** du bloc `if` représentant le cas où il y a un problème, c'est ici que l'on pourrait souhaiter **lever une exception**. Il nous faut d'abord créer une exception correspondant à notre cas problématique, c'est à dire :

* Créer une classe vide héritant de `Exception`, et au nom explicite représentant notre cas d'erreur
* Lever une instance de cette exception, au moment où le cas d'erreur visé survient dans notre code

In [11]:
# Déclaration de la classe de notre exception
class TemperatureTooHot(Exception):
    pass

print("Ouverture de l'appareil photo du téléphone.")
temperature = 42

if temperature >= 40:
    # Levée d'une nouvelle instance de notre exception
    raise TemperatureTooHot("La température extérieure est trop élevée.")
else:
    print("Cheese !")

Ouverture de l'appareil photo du téléphone.


TemperatureTooHot: La température extérieure est trop élevée.

L'intérêt des exceptions est que, même levées dans une fonction qui appelle une fonction qui en appelle une autre, on pourra toujours finir par la capturer grâce au bloc `try/except` vu dans le chapitre de cours précédent.

Voyons un exemple avec un enchaînement de fonctions imbriquées, et une exception qui est levée au sein de l'une d'entre elles.

In [ ]:
class PaymentBadCurrency(Exception):
    pass

# 3ème fonction appelée
def verifier_paiement(devise):
    print("3. Vérification du paiement...")
    if devise != "euro":
        # Levée de l'exception
        raise PaymentBadCurrency("On n'accepte que les paiements en euro !")

# 2ème fonction appelée
def gerer_commande(devise):
    print("2. Gestion de la commande...")
    verifier_paiement(devise)

# 1ère fonction appelée
def lire_formulaire(devise):
    print("1. Lecture du formulaire...")
    # Avant d'appeler la 2ème fonction, on se prépare à la capture
    # d'une éventuelle exception plus loin dans le code
    try:
        gerer_commande(devise)
    except PaymentBadCurrency as err:
        print(f"Le paiement a échoué sur la devise : {err}")
        return "Code HTTP 500 (Internal Server Error)"
    
    print("Formulaire de paiement géré avec succès.")
    return "Code HTTP 200 (OK)"

# Remplacez "yen" par "euro" pour tester un comportement sans erreur
lire_formulaire("yen")

1. Lecture du formulaire...
2. Gestion de la commande...
3. Vérification du paiement...
Le paiement a échoué sur la devise : On n'accepte que les paiements en euro !


'Code HTTP 500 (Internal Server Error)'

---
## 🖊️ La levée d'exceptions de debug avec `assert`

Il existe une façon simplifiée de lever immédiatement une exception afin d'arrêter un programme car il n'y a pas de sens d'aller plus loin. Cependant, il ne faut s'en servir qu'à des fins de **débogage** ou lors de **tests unitaires**. Tout code voulant proprement gérer ses exceptions doit les lever avec `raise` et les capturer avec `try/except`.

Cette façon simplifiée utilise le mot-clé `assert` qui sera suivi d'une expression booléenne représentant un cas où tout se passe bien. Mais si lors de l'exécution du code, l'expression s'avère fausse, alors Python lèvera une exception de type `AssertionError` (classe héritant de `Exception`).

Son usage se fait sous la forme `assert [expression booléenne]`. Sur l'exemple précédent, vérifions un cas qui est censé être rare mais que l'on devrait peut-être vérifier lors du développement d'une application de photographie : la présence ou l'absence d'appareil photo sur un téléphone.

In [ ]:
print("Ouverture de l'appareil photo du téléphone.")
hasBackCamera = False

assert hasBackCamera == True

print("Cheese !")

Ouverture de l'appareil photo du téléphone.


AssertionError: 

L'erreur affichée par Python vous montre clairement où s'est déroulée l'exception, ainsi que son type `AssertionError`.

Pour être plus utile au développeur, on peut préciser un message qui puisse accompagner cette exception : il suffit de l'écrire juste après l'expression booléenne du `assert`.

On écrit donc finalement `assert [expression booléenne], [message de l'exception levée]`

In [ ]:
print("Ouverture de l'appareil photo du téléphone.")
hasBackCamera = False

assert hasBackCamera == True, "Le téléphone n'a pas d'appareil photo"

print("Cheese !")

Ouverture de l'appareil photo du téléphone.


AssertionError: Le téléphone n'a pas d'appareil photo

---
## 🧰 Bibliothèque standard - `os.path`

Cette petite bibliothèque sert à réaliser de nombreuses opérations en rapport avec des **chemins** de dossiers ou de fichiers.

Bien que cela peut paraître trivial, souvenez-vous que les systèmes d'exploitation ont différents séparateurs au sein des chemins de dossiers : `\` pour Windows, `/` pour Unix (et donc aussi Linux et macOS)... d'où l'utilité d'avoir un moyen de toujours utiliser le bon séparateur qu'importe l'OS sur lequel tourne notre code Python.

Parmi toutes les fonctions de cette bibliothèque, voici les plus utiles :

* `.join()` : construit un chemin en utilisant le bon séparateur du système d'exploitation actuel
* `.exists()` : renvoie si le chemin passé est un fichier ou dossier qui existe
* `.isfile()` : renvoie si le chemin passé est un fichier
* `.isdir()` : renvoie si le chemin passé est un dossier
* `.splitext()` : à partir d'un chemin, renvoie séparément le chemin complet du fichier, puis l'extension du fichier

In [2]:
import os

testpath = os.path.join("assets", "testfile.txt")
print(testpath)
print(f"Existe ? {os.path.exists(testpath)}")
print(f"Fichier ? {os.path.isfile(testpath)}")
print(f"Dossier ? {os.path.isdir(testpath)}")
filename, ext = os.path.splitext(testpath)
print(f"Le fichier {filename} a pour extension {ext}")

assets\testfile.txt
Existe ? True
Fichier ? True
Dossier ? False
Le fichier assets\testfile a pour extension .txt


---

## 🧰 Bibliothèque standard - `datetime`

Depuis très longtemps, l'informatique a choisi de stocker facilement une date et une heure donnée en comptant le nombre de secondes écoulées depuis ce qu'on appelle l'[**époque Unix**](https://fr.wikipedia.org/wiki/Heure_Unix) située au 1er janvier 1970 à minuit (UTC). Mais en plus de ne pas être une mesure très rigoureuse, le stockage d'un nombre de secondes sous-entends le fuseau horaire UTC sans réellement l'expliciter, et cela peut porter à confusion lors de son usage dans une base de donnes ou dans une application.

Autant pour facilier la gestion des dates que pour assurer une rigueur dans leur usage, Python propose la bibliothèque standard `datetime`, avec laquelle on utilise des objets et des méthodes afin de manipuler proprement et facilement des **dates** et des **heures**.

Cette dernière propose notamment des moyens de représenter facilement soit juste une date, soit juste un horodatage, soit les deux. Voyons tout d'abord comment avoir la date ou l'horodatage du moment présent :

In [3]:
# Imports depuis la bibliothèque standard datetime
from datetime import date, datetime

date_instance = date.today() # 2024-04-30
datetime_instance = datetime.now() # 2024-04-30 15:22:55.996496

print(date_instance)
print(datetime_instance)

2024-04-30
2024-04-30 15:22:55.996496


Grâce à ces fonctions, vous rencontrez déjà des instances des classes `date` et `datetime` sur lesquels on peut réaliser énormément d'opérations.

On peut accéder très simplement à une valeur en particulier, par exemple :

* Sur les `date` et `datetime` : `.day` pour le jour, `.month` pour le mois, `.year` pour l'année
* Sur les `time` et `datetime` : `.hour` pour les heures, `.minute` pour les minutes, `.second` pour les secondes

In [5]:
print(f"Nous sommes en l'année {date.today().year}.")
print(f"Il est {datetime.now().hour} heures et {datetime.now().minute} minutes.")

Nous sommes en l'année 2024.
Il est 15 heures et 25 minutes.


Il est également possible de créer un de ces objets en passant chaque valeur une à une, comme ci-dessous où l'on va créer une instance de chaque classe.

In [6]:
from datetime import date, time, datetime

# Création d'une date
date_instance = date(year=2022, month=7, day=14)
# Création d'un horaire
time_instance = time(hour=13, minute=14, second=31)
# Création d'une date avec horaire
datetime_instance = datetime(year=2020, month=1, day=31, hour=13, minute=14, second=31)

print(date_instance)
print(time_instance)
print(datetime_instance)

2022-07-14
13:14:31
2020-01-31 13:14:31


Vous avez sûrement constaté que l'affichage de ces types, une fois implicitement convertis en `str` par la fonction `print()`, est assez rudimentaire.

Il existe un moyen de choisir comment ces types seront convertis en chaîne de caractères grâce au **formatage**, c'est à dire indiquer comment afficher "joliment" une date et/ou un horaire.

Sur la [documentation Python](https://docs.python.org/fr/3/library/datetime.html#strftime-and-strptime-format-codes) se trouve une liste de ce qu'on appelle des **directives**, qui servent à indiquer qu'à tel endroit d'une chaîne de caractères, l'on souhaite avoir le numéro d'un jour ou le nom du mois correspondant à une date.

Le numéro du jour dans le mois s'affiche donc avec la directive `%d`, et l'année sur 4 chiffres s'affiche avec `%Y`, etc.

Il suffit donc de passer une chaîne de caractères contenant ces **directives** à la méthode `.strftime()` sur l'instance d'un classe proposée par la bibliothèque `datetime`, et les directives seront remplacées par des valeurs.

In [7]:
from datetime import datetime

datetime_instance = datetime.now()
print(datetime_instance.strftime("%A %d %B, à %H:%M:%S"))

Tuesday 30 April, à 17:27:03


On peut utiliser ces mêmes **directives** lorsque l'on veut rapidement créer une date/horaire à partir d'une chaîne de caractères qui contient une date et/ou un horaire dans un certain sens, afin d'indiquer à Python où se trouvent le jour, le mois, l'année, l'heure, les minutes... dans une chaîne de caractères.

La fonction `datetime.strptime()` s'occupera de réaliser la conversion vers une classe de `datetime`.

In [54]:
from datetime import datetime

created_at = "13/04/2021 13:37:10"
converted_created_at = datetime.strptime(created_at, "%d/%m/%Y %H:%M:%S")
print(converted_created_at)

2021-04-13 13:37:10


Cependant, lorsqu'une date respecte le format [ISO 8601](https://fr.wikipedia.org/wiki/ISO_8601), il n'y a pas besoin de préciser au module `datetime` comment comprendre une date écrite dans une chaîne de caractères.

L'usage de la fonction `datetime.fromisoformat()` servira à obtenir directement une instance de la classe `datetime` dans ce cas précis.

In [3]:
from datetime import datetime

event_start = "2023-07-13T19:30:00+12:00"
kickoff = datetime.fromisoformat(event_start)
print(f"Le match démarre à {kickoff.hour}:{kickoff.minute}.")


Le match démarre à 19:30


Il existe également un moyen d'additionner ou soustraire des valeurs à l'aide du type [timedelta](https://docs.python.org/fr/3/library/datetime.html#timedelta-objects), qui représente uniquement **une durée**.

Par exemple, si l'on veut savoir quel jour il sera dans 2 semaines, ou dans 48 heures, etc... il suffit d'instancier le type `timedelta` avec la durée souhaitée, puis de le soustraire ou de l'additionner à autre chose.

In [12]:
from datetime import timedelta

# Création d'une durée de 2 semaines
migration_duration = timedelta(weeks=2)

# Addition de la durée de 2 semaines au moment présent
migration_end = date.today() + migration_duration

print(f"Nous sommes le {date.today().strftime('%d/%m/%Y')}.")
print(f"La migration de votre serveur sera terminée d'ici le {migration_end.strftime('%d/%m/%Y')}.")

Nous sommes le 30/04/2024.
La migration de votre serveur sera terminée d'ici le 14/05/2024.


Mais il est également possible de soustraire des dates entre elles afin d'en obtenir l'**intervalle** entre elles. Par exemple, soustraire une date future et aujourd'hui permet d'obtenir le nombre de jours restant avant d'y arriver.

In [ ]:
from datetime import date

countdown = date(day=26, month=7, year=2024) - date.today()

print(f"Il reste {countdown.days} jours avant les jeux de Paris.")

Il reste 836 jours avant les jeux de Paris.


---

## 🧰 Bibliothèque standard - `json`

Né du monde JavaScript, le format **JSON** s'est plus ou moins imposé face au XML comme étant un format d'échange de données plus léger que ce dernier, et surtout plus pratique à implémenter dans les langages qui souhaitent l'utiliser.

Techniquement, JSON est un "sous-ensemble limité" du langage JavaScript, d'où le fait que son écriture ressemble à ce dernier tout en ayant des restrictions (absence des commentaires...). De ce fait, vous pouvez écrire un objet JSON au milieu d'un code JavaScript et le manipuler directement.

Ce format est donc fréquemment utilisé pour échanger des informations de manière structurée avec des API, mais on peut également s'en servir pour stocker des données dans des fichiers sous un format lisible autant par les humains que par des applications. On parle alors de "document JSON", sans que l'extension de fichier `.json` ne soit forcément imposée. La preuve, le notebook Jupyter que vous êtes en train de consulter est en réalité un document JSON !

Lorsque l'on travaille avec ce format dans un autre langage que JavaScript, on doit forcément convertir les données au format JSON en données lisibles par le langage que nous utilisons. L'action de conversion depuis le format JSON se dit **désérialiser**, et la conversion vers le format JSON se dit **sérialiser**.

Il y a peu de méthodes sur cette bibliothèque, mais leur orthographe peut semer la confusion :

* `.load()` déserialise du JSON en Python, à partir d'un fichier ouvert (avec `open()` par exemple)
* `.dump()` sérialise du Python en JSON, dans un fichier ouvert (avec `open()` par exemple)
* `.loads()` déserialise du JSON en Python, à partir d'une chaîne de caractères
* `.dumps()` sérialise du Python en JSON, dans une chaîne de caractères

Les méthodes au **singulier** s'utilisent donc avec des fichiers, quand celles au **pluriel** s'utilisent avec des chaînes de caractères.

Importons la bibliothèque puis effectuons une **sérialisation** toute simple d'une valeur Python vers du JSON :

In [13]:
import json

serialized = json.dumps(None)

print(serialized)
print(type(serialized))

null
<class 'str'>


On obtiens donc la chaîne de caractères "null", et l'on voit bien que le type de valeur en Python a été correctement "traduite" vers le type correspondant en JavaScript.

`True` deviendra par exemple `true`, tout en minuscules. Pour le reste des valeurs, comme les nombres, les listes ou dictionnaires, leur écriture en JSON est assez similaire au Python.

Faisons désormais la sérialisation vers du JSON d'une valeur Python un peu plus complexe :

In [ ]:
import json

py_list = [
    {
        "name": "john",
        "year": 2013,
        "is_debug": True
    }
]

json_serialized = json.dumps(py_list)

print(json_serialized)

[{"name": "john", "year": 2013, "is_debug": true}]


Ici, toute la liste et son contenu ont été converties en valeurs JSON, et réduites en une simple ligne (qui est toujours une chaîne de caractères). On peut donc ensuite très facilement la transmettre ou encore la sauvegarder quelque part.

Procédons maintenant à l'opération inverse, c'est à dire la **désérialisation** d'un objet JSON vers du Python.

In [ ]:
import json

# Pour rappel, les trois guillemets permettent d'écrire du texte multi-lignes
json_data = """
{
    "site_url": "www.superwebsite.dev",
    "site_port": 8080,
    "debug_mode": false
}
"""

python_data = json.loads(json_data)

print(python_data)
print(python_data["site_url"])

{'site_url': 'www.superwebsite.dev', 'site_port': 8080, 'debug_mode': False}
www.superwebsite.dev


Après la **déserialisation** de cet objet JSON, qui était un dictionnaire, on obtient également un dictionnaire Python et l'on peut s'en servir directement en tant que tel. Toute modification réalisée en Python ne sera pas répliquée sur l'objet JSON original et vice-versa : il faudra à nouveau sérialiser ou désérialiser au moment voulu.

Pour ce qui est des méthodes au singulier `json.load()` et `json.dump()`, elles nécessitent donc d'être utilisées avec un fichier qui est ouvert. C'est ce que nous permet la fonction Python built-in `open()` vue au dernier chapitre !

Étant donné que l'on souhaite généralement stocker plusieurs valeurs dans un fichier, on utilisera alors un dictionnaire dans lequel on rangera toutes les valeurs souhaitées.

In [14]:
import json

# Construction du chemin vers le fichier à lire
path = os.path.join("assets", "testdata.json")
# On déclare la variable en dehors du scope du "with" pour pouvoir y accéder
python_data = None

# Ouverture d'un fichier...
with open(path, "r", encoding="utf-8") as f:
    # Lecture et conversion du JSON en même temps
    python_data = json.load(f)

print(python_data)

for city in python_data['cities']:
    print(f"\n{city['name_latin']} a une population de {city['population']} habitants.")

{'cities': [{'name_ja': '東京都', 'name_latin': 'Tōkyō', 'population': 13960236}, {'name_ja': '大阪市', 'name_latin': 'Osaka', 'population': 2751862}, {'name_ja': '福岡市', 'name_latin': 'Fukuoka', 'population': 1603043}]}

Tōkyō a une population de 13960236 habitants.

Osaka a une population de 2751862 habitants.

Fukuoka a une population de 1603043 habitants.


Quant à la fonction `json.dump()`, on peut lui passer la valeur Python que l'on souhaite sérialiser ainsi que le fichier de destination que l'on vient d'ouvrir en mode écriture.


In [ ]:
import json

# Construction du chemin vers le fichier à lire
path = os.path.join("..", "assets", "serialized.json")
# Dictionnaire Python qui sera converti plus tard en JSON
python_data = {
    "languages": ["Python", "Java", "C#", "JavaScript", "HTML", "CSS"]
}

# Le mode "w" (write) va écrire dans le fichier en remplaçant tout son contenu
with open(path, "w", encoding="utf-8") as f:
    # Conversion et écriture du JSON en même temps
    json.dump(python_data, f)

Vous pouvez désormais ouvrir le fichier `serialized.json` du dossier `assets` pour y voir nos données converties au format JSON.

---

## ⚙️ Installer un paquet avec PIP

Lorsque l'on souhaite utiliser des projets tiers qui ne sont pas inclus dans le langage que l'on utilise, il faudra toujours installer ce qu'on appelle des **"paquets"**, et ce à l'aide d'un **gestionnaire de paquets**.

Dans le cas de Python, l'intégralité des paquets installables sont recensés sur [PyPI](https://pypi.org/), et le gestionnaire de paquets s'appelle **PIP**.

Contrairement à d'autres langages, un paquet installé en Python sera installé "globalement", c'est à dire que le paquet sera lié à votre installation de Python plutôt qu'à votre projet ou répertoire en cours. Nous verrons plus tard une façon de résoudre ce problème. 

PIP s'appelle dans un terminal comme n'importe quelle autre application, et possède plusieurs commandes pour gérer les paquets tiers. En voici quelques-unes :

* `pip list` : liste tous les paquets installés
* `pip install [paquet]` : installe un paquet
* `pip uninstall [paquet]` : désinstalle un paquet 

Ainsi, si vous souhaitez installer le paquet `Pillow`, il suffira d'écrire `pip install Pillow`

### Le fichier requirements.txt

Pour garder une trace des paquets tiers dont on a besoin dans un projet, on utilise un fichier `requirements.txt` qui peut être écrit et lu par PIP. Combiné aux environnements virtuels, c'est donc quelque chose d'extrêmement pratique. 

Après avoir installé les paquets nécessaires à votre projet au sein d'un environnement virtuel, vous pouvez rapidement exporter la liste de ces paquets puis l'écrire dans le fichier `requirements.txt` en exécutant ceci :

`pip freeze > requirements.txt`

Techniquement, cela va y inscrire la version exacte des paquets installés, afin d'être certain d'assuré la compatibilité avec votre code.

Ensuite, lorsqu'il sera l'occasion de devoir installer tous les paquets nécessaires au projet, la commande `install` pourra lire ce fichier de cette façon :

`pip install -r requirements.txt`

---

## ⚙️ Les environnements virtuels

Derrière ce nom se cache un moyen d'avoir facilement **une copie de votre installation de Python** afin de travailler de façon "isolée". Le premier avantage d'utiliser cette installation isolée, que l'on appelle un **environnement virtuel**, est que tous les paquets installés avec PIP y seront exclusifs.

Ainsi, on évite toutes les contraintes liées au fait que les paquets soient d'habitude installés de façon globale, qu'importe l'endroit sur votre ordinateur duquel vous appelez Python.

Par exemple, si vous avez un projet nécessitant le paquet `Django` en version 2.0, et un autre projet qui en ait besoin dans sa version 3.0, le fait que chaque projet possède alors son propre **environnement virtuel** permettra d'y installer juste les paquets nécessaires dans la version exacte nécessaire à leur bon fonctionnement.

Bien que l'on ait l'habitude d'utiliser un projet tiers nommé `virtualenv` pour créer ces environnements virtuels, le module `venv` est désormais inclus dans Python à partir de la version 3.4 du langage : nous utiliserons donc ce dernier par simplicité.



### Créer un environnement virtuel

La création d'un environnement virtuel se fait dans un terminal avec la commande suivante :

`python -m venv [nom]` 

Une fois situé dans le dossier racine de votre projet, qui s'appelle par exemple `projet_python`, la création d'un envionnement nommé par exemple "monenv" se fera alors comme ceci :

`python -m venv monenv` 

![Arborescence d'un projet avec venv](assets/venv.svg)

Comme indiqué sur l'image ci-dessus, votre environnement prend simplement la forme d'un dossier. C'est là-dedans que sont stockées les informations relatives à votre environnement virtuel, les paquets qui y seront installés, etc.

Pour information, ce dossier n'est pas à inclure dans un dépôt Git : il faut laisser les personnes (ou les scripts automatisés) qui accèdent au projet le soin de créer eux-mêmes un environnement virtuel.


### Utiliser un environnement virtuel

Pour utiliser un environnement virtuel, on dit que l'on va **activer** ce dernier.

Toujours dans un terminal, le fait d'exécuter un certain fichier fera en sorte que l'on soit désormais "dans" l'environnement virtuel lorsque l'on fera appel plus tard à `python` ou à `pip`.

Le fichier d'activation n'est pas le même selon l'invite de commande que vous utilisez :


* Pour l'invite de commande **Windows par défaut** (cmd)

`.\monenv\Scripts\activate.bat`

* Pour l'invite de commande **Windows PowerShell**

`.\monenv\Scripts\activate.ps1`

* Pour un invite de commande **bash** et dérivés (Linux, macOS, WSL...)

`source .\monenv\Scripts\activate`

![Activation dans des terminaux](assets/venv_activate.png)

Tel que montré ci-dessus, que notre environnement virtuel soit activé dans **PowerShell** ou dans **cmd**, on voit visuellement la différence avec l'ajout du nom de notre environnement au début de chaque nouvelle ligne. Cela permet de nous souvenir que nous sommes actuellement "dans" notre environnement virtuel.

<details>
  <summary>【💡 Spoiler】</summary>
  En réalité, l'activation simplement va faire en sorte que, lorsque vous appelez python ou pip dans votre terminal, ce dernier aille utiliser les exécutables correspondants dans le dossier de votre environnement, plutôt que d'utiliser les exécutables installés sur votre système d'exploitation.
</details>

Pour **"sortir"** de votre environnement virtuel, vous pouvez bien sûr soit quitter votre invite de commande, soit taper tout simplement la commande suivante :

`deactivate`

![Désactivation dans un terminal](assets/venv_deactivate.png)

Vous pouvez ci-dessus constater visuellement la sortie de l'environnement virtuel.

---

## 📚 Bibliothèque tierce - Pillow

Maintenant que nous savons installer des paquets pour utiliser des projets tiers, essayons d'utiliser le projet [**Pillow**](https://pillow.readthedocs.io/en/stable/index.html). C'est un dérivé moderne d'une ancienne bibliothèque nommée "PIL" visant à manipuler des images en Python. Aujourd'hui, Pillow est une bibliothèque de référence lorsqu'il s'agit de lire et manipuler des images.

Pour l'installer, ouvrez un invite de commandes sur votre ordinateur et tapez `pip install Pillow`.
Fermez puis rouvrez ce cours lorsque cela sera fait.

Commençons à utiliser la bibliothèque par un premier exemple très simple : ouvrir une image, réduire son nombre de couleurs, puis l'afficher.

In [24]:
import os
from PIL import Image

# Construction du chemin vers l'image à l'aide de os.path
image_path = os.path.join("assets", "osaka.jpg")

# Ouverture de l'image, qui sera utilisable en tant que variable "im"
with Image.open(image_path) as im:
    # Réduction des couleurs, puis affichage
    im.quantize(colors=30).show()

Une fois une image ouverte avec Pillow, on a alors une instance de la classe `Image` sur laquelle de nombreuses fonctions sont disponibles pour la modifier ou en obtenir des informations. Référez-vous à sa page dans la [**documentation**](https://pillow.readthedocs.io/en/stable/reference/Image.html) pour avoir une liste exhaustive de ce que l'on peut faire.

Bien sûr, la méthode `.show()` affichant une image n'est à utiliser qu'en conditions de test comme ici. En temps normal, on voudra plutôt sauvegarder notre image après l'avoir modifiée, et idéalement sous un autre nom pour ne pas écraser l'image originale. Voyons cela.

In [19]:
import os
from PIL import Image

# Construction du chemin vers l'image à l'aide de os.path
path = os.path.join("assets", "osaka.jpg")

with Image.open(path) as im:
    # Réduction de l'image à une taille de 256*256 pixels maximum
    im.thumbnail((256, 256))
    # "filename" recevra le chemin et le nom du fichier, "ext" recevra juste l'extension
    filename, ext = os.path.splitext(path)
    # Le nom du nouveau fichier utilise le chemin et l'extension du fichier original
    destination = f"{filename}_thumbnail{ext}"
    # Sauvegarde de l'image au format JPEG
    im.save(destination, "JPEG")

Grâce à une méthode `.thumbnail()`, nous avons rapidement créé une version miniature d'une image existante. C'est quelque chose qui pourra être utile si vous êtes amené à gérer un upload d'image, par exemple.

Une autre façon de manipuler nos images est de **dessiner** par-dessus. Pour cela, il faudra importer le module `ImageDraw` pour y passer l'image sur laquelle on souhaite dessiner, et utiliser [ses nombreuses méthodes](https://pillow.readthedocs.io/en/stable/reference/ImageDraw.html).

Avant de continuer, il faut d'abord comprendre le passage de coordonnées lorsque l'on appelle les méthodes de ce module. Pour dessiner un rectangle par exemple, au lieu de donner des coordonnées de départ puis la largeur et la hauteur du rectangle souhaité, on passe d'abord des **coordonnées de départ** puis des **coordonnées d'arrivée**, le tout dans un seul **tuple**.

Pour ce qui est de l'**origine des coordonnées** d'ailleurs, elle se situe toujours en haut à gauche.

![Explication des coordonnées](assets/pillow.svg)

Dessinons un rectangle à un endroit précis et avec une couleur précise.

In [50]:
import os
from PIL import Image, ImageDraw


# Construction du chemin vers l'image à l'aide de os.path
image_path = os.path.join("assets", "osaka.jpg")

# Ouverture de l'image, qui sera utilisable en tant que variable "im"
with Image.open(image_path) as im:
    # Réduction des couleurs, puis affichage
    # On transmet notre image à une classe permettant d'y dessiner dessus
    draw = ImageDraw.Draw(im)
    # Le premier argument est un tuple indiquant les positions x et y de départ,
    # puis les positions x et y d'arrivée. Pour la couleur de remplissage (fill),
    # il faut l'écrire avec un tuple au format RGB (rouge, vert, bleu)
    draw.rectangle((280, 150, 700, 220), fill=(0, 126, 72))
    
    # Afficher l'image (J'ajoute rotate(0) car j'ai une erreur sans)        
    im.rotate(0).show()

Une autre méthode très utile de `ImageDraw` nous permet d'écrire le texte de notre choix sur une image, en précisant également sa couleur.

In [51]:
import os
from PIL import Image, ImageDraw

path = os.path.join("assets", "osaka.jpg")

color_white = (255, 255, 255)
color_black = (0, 0, 0)

with Image.open(path) as im:
    draw = ImageDraw.Draw(im)
    draw.rectangle((280, 150, 700, 220), fill=color_white)
    # Une position et un texte suffisent à écrire sur l'image
    draw.text((400, 180), "Fukuoka", fill=color_black)
    im.rotate(0).show()

Bien que la fonction d'écriture de texte fonctionne, la police par défaut est assez petite, et sans spécialement très lisible belle non plus.

Pour cela, le module `ImageFont` de Pillow permet d'utiliser un **fichier de police d'écriture** afin d'écrire du texte avec.

Le seul petit inconvénient est qu'il faille toujours ouvrir un fichier de police avec une taille prévue à l'avance : si vous voulez donc utiliser la même police à des tailles différentes, il faudra la préparer plusieurs fois à l'avance avec plusieurs tailles.

In [53]:
import os
from PIL import Image, ImageDraw, ImageFont

path = os.path.join("assets", "osaka.jpg")
path_font = os.path.join("assets", "OpenSans-Bold.ttf")

color_white = (255, 255, 255)
color_black = (0, 0, 0)

# On prépare l'usage d'une police, en précisant la taille souhaitée qui est ici à 42
font_opensans_42pt = ImageFont.truetype(path_font, 42)

with Image.open(path) as im:
    draw = ImageDraw.Draw(im)
    draw.rectangle((280, 150, 700, 220), fill=color_white)
    # On écrit le texte en précisant la police, et toujours la couleur
    draw.text((400, 155), "Fukuoka", font=font_opensans_42pt, fill=color_black)
    im.rotate(0).show()

---

# Exercice

Une fois ce cours terminé, vous pourrez réaliser l'exercice du dossier `rugby`.